In [45]:
import matplotlib.pyplot as plt
import numpy as np 
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
import pandas as pd
import matplotlib.patches as patches
from PIL import Image
import pandas as pd
import cv2
import time as t
import os
import shutil
import copy

In [1]:
class Measurment(object):
    
    def __init__(self, path_to_data, zone_coordinates, zone_size, skiped_img,   path_to_result = None, ):
        self.first_point_coord_x = zone_coordinates[0][0]  
        self.first_point_coord_y = zone_coordinates[0][1] 
        self.second_point_coord_x = zone_coordinates[1][0]
        self.second_point_coord_y = zone_coordinates[1][1]
        self.third_point_coord_x = zone_coordinates[2][0]
        self.third_point_coord_y = zone_coordinates[2][1]
        self.zone_size = zone_size
        self.skiped_img = skiped_img
        self.path_to_data = self.path_data(path_to_data)
        self.path_to_resault = self.path_res(path_to_result)
        self.img_names = os.listdir(self.path_to_data)
        self.first_point_data, self.second_point_data, self.third_point_data = self.get_data()
        
        
        
        
    def  calculate_params (self, interval):
        start_time = t.time()
        self.intrval = interval
        data = [self.first_point_data, self.second_point_data, self.third_point_data]
        self.calculated_time_param = [] 
        self.calculated_value_param = []
        self.max_point_data = []
        for point in range(3): 
            self.max_point_data.append(self.max_research(data[point], interval))
            front_01_value = []
            front_09_value = []
            back_09_value = []
            back_01_value = []
            front_09_time = []
            front_01_time = []
            back_09_time = []
            back_01_time = []
            iteration = 0
            frame_controller = 0
            num_of_pulses = len(self.max_point_data[point])
            puls = 0
            last_time = 0
            last_value = 0
            controller = 0
            controller_for_interval = 0
            controller_front_01 = 0
            controller_front_09 = 0
            controller_back_01 = 0
            controller_back_09 = 0
            for time, value  in  enumerate(data[point]):
                if puls == num_of_pulses:
                    break
                if (
                    value >= self.max_point_data[point][puls] * 0.1 
                    and last_value <= self.max_point_data[point][puls] 
                    and controller_front_01 == 0
                    and frame_controller == 0
                    ):
                    error_01_front_value = value - self.max_point_data[point][puls] * 0.1 
                    error_01_front_last_value = self.max_point_data[point][puls] * 0.1  - value
                    if error_01_front_value <=  error_01_front_last_value:
                        best_value_01_front = value
                        best_time = time
                    else:
                        best_value_01_front = last_value
                        best_time = last_time
                    
                    controller_front_01 += 1
                    controller += 1
                    frame_controller += 1
                    front_01_value.append(best_value_01_front)
                    front_01_time.append(best_time)
                elif (
                      value >= self.max_point_data[point][puls] * 0.9 
                      and last_value <= self.max_point_data[point][puls] * 0.9 
                      and controller_front_09 == 0
                      and frame_controller == 0
                      ):
                    error_09_front_value = value - self.max_point_data[point][puls] * 0.9
                    error_09_front_last_value = self.max_point_data[point][puls] * 0.9  - value
                    if error_09_front_value <=  error_09_front_last_value:
                        best_value_09_front = value
                        best_time = time
                    else:
                        best_value_09_front = last_value
                        best_time = last_time
                    controller_front_09 += 1
                    controller += 1
                    frame_controller += 1
                    front_09_value.append(best_value_09_front)
                    front_09_time.append(best_time)
                elif (
                      value <= self.max_point_data[point][puls] * 0.9 
                      and last_value >= self.max_point_data[point][puls] * 0.9 
                      and controller_back_09 == 0
                      and frame_controller == 0
                      ):
                    error_09_back_value = self.max_point_data[point][puls] * 0.9 - value
                    error_09_back_last_value = value - self.max_point_data[point][puls] * 0.9
                    if error_09_back_value <=  error_09_back_last_value:
                        best_value_09_back = value
                        best_time = time
                    else:
                        best_value_09_back = last_value
                        best_time = last_time                        
                    controller_back_09 += 1
                    controller += 1
                    frame_controller += 1
                    back_09_value.append(best_value_09_back)
                    back_09_time.append(best_time)
                elif (
                      value <= self.max_point_data[point][puls] * 0.1 
                      and last_value >= self.max_point_data[point][puls] * 0.1 
                      and controller_back_01 == 0
                      and frame_controller == 0
                     ):
                    error_01_back_value = self.max_point_data[point][puls] * 0.1 - value
                    error_01_back_last_value = value - self.max_point_data[point][puls] * 0.1
                    if error_01_back_value <=  error_01_back_last_value:
                        best_value_01_back = value
                        best_time = time
                    else:
                        best_value_01_back = last_value
                        best_time = last_time                        
                    controller_back_01 += 1
                    controller += 1
                    frame_controller += 1
                    back_01_value.append(best_value_01_back)
                    back_01_time.append(best_time)
                    
#                 print(frame_controller, iteration, controller, controller_for_interval, controller_front_01,controller_front_09,controller_back_09,controller_back_01)
                
                
                
                if frame_controller != 0:
                        iteration += 1
                        if  iteration == 20:
                            frame_controller = 0
                            iteration = 0
                
                
                if controller == 4:
                    controller_for_interval += 1
                    if controller_for_interval == 100:
                        puls += 1
                        controller_for_interval = 0
                        controller_front_01 = 0
                        controller_front_09 = 0
                        controller_back_09 = 0
                        controller_back_01 = 0
                        controller = 0
                    
                    
                last_value = copy.copy(value)
                last_time = copy.copy(time)
            
            
            time = [
                    np.array(front_01_time), np.array(front_09_time),
                    np.array(back_09_time), np.array(back_01_time)
                    ]
            value = [np.array(front_01_value),np.array(front_09_value),
                     np.array(back_09_value),np.array(back_01_value)
                    ]
            
            self.calculated_time_param.append(time)
            self.calculated_value_param.append(value)
        self.calculate()
        print("The parameters data is calculated  --- %s seconds ---" % (t.time() - start_time))
            
            
        
    def calculate(self):
        self.full_front = np.array([])
        self.full_back = np.array([])
        self.full_width  = np.array([])
        self.full_period = []
        full_09_front = np.array([])
        full_09_back =  np.array([])
        for point in range(3):
            
            
            front_time = self.calculated_time_param[point][1] - self.calculated_time_param[point][0]
            back_time = self.calculated_time_param[point][3] - self.calculated_time_param[point][2]
            impuls_width = self.calculated_time_param[point][2] - self.calculated_time_param[point][1]
            
            full_09_back = np.hstack([full_09_back, self.calculated_time_param[point][2]])
            self.full_front = np.hstack([self.full_front, front_time])
            self.full_back = np.hstack([self.full_back, back_time])
            self.full_width = np.hstack([self.full_width, impuls_width])
        
        
        self.num_of_pulses = sum([len(element) for element in self.max_point_data])  
        i = 0
        full_09_back = np.sort(full_09_back)
        while i < self.num_of_pulses - 1:
            self.full_period.append(full_09_back[i + 1] - full_09_back[i])
            i += 1

        
        
        print("Number of front times = ", len(self.full_front))
        print("Number of back times  = ", len(self.full_back))
        print("Numper of pulse's widths = ", len(self.full_width))
        print("Number of periods  = ", len(self.full_period))
        print("Numper of pulses = ", self.num_of_pulses )
        print("For getting  the parameters, use the method 'get_params(confidence probability)'")
        print("For getting list of parameters, use attribute 'params' ")
        self.params = [self.full_front, self.full_back, self.full_width, self.full_period]
        
        
        
    
    def get_params(self,conf_prob):
        parametrs_char = []
        for params in self.params:
             parametrs_char.append(self.error(params, conf_prob))
        names = ["FRONT", "BACK", "WIDTH" , "PERIOD"]
                
        print ("                              Time parameters  of SLM                        ")
        
        for i in range(4):
            print(" {}  Mean = {}    NSTD = {}   VAR = {}  CONFIDENCE INTERVAL = {}   CONFIDENCE PROBABILITY {}  ".format(names[i],
                str(parametrs_char[i][0]), str(parametrs_char[i][1]), str(parametrs_char[i][2]), str(parametrs_char[i][3]), conf_prob)
                )
                                                                                                                   


    def error(self, data, conf_prob):
        mean = np.mean(data)
        n = len(data)
        from scipy.stats import t 
        student_coef = round(t.ppf((1 + conf_prob )/2, n - 1), 2)
        var = np.var(data)
        interval = student_coef * math.sqrt((var)/n) 
        nskd = math.sqrt(var)
        return np.around(np.array([mean, nskd,  var, interval]),decimals=3 )
        
        
        
        
    
 
       
    def get_data(self):
        start_time = t.time()
        first_point = []
        second_point = []
        third_point = []
        imgs_min = []
        for i in range(self.skiped_img, len(self.img_names)):
            img = np.array(Image.open(self.path_to_data + "\\" + self.img_names[i]))
            first_point_zone = img[
                                    self.first_point_coord_x:self.first_point_coord_x + self.zone_size,
                                    self.first_point_coord_y:self.first_point_coord_y + self.zone_size
                                   ]
            second_point_zone = img[
                                    self.second_point_coord_x:self.second_point_coord_x + self.zone_size,
                                    self.second_point_coord_y:self.second_point_coord_y + self.zone_size
                                    ]
            third_point_zone = img[
                                    self.third_point_coord_x:self.third_point_coord_x + self.zone_size,
                                    self.third_point_coord_y:self.third_point_coord_y + self.zone_size
                                   ]
            
            first_point.append(np.sum(first_point_zone))
            second_point.append(np.sum(second_point_zone))
            third_point.append(np.sum(third_point_zone))       
            img_min = np.min(img)
            imgs_min.append(img_min)
        mean_img_min = np.mean(np.array(imgs_min))    
        first_point = np.array(first_point) -  self.zone_size**2*mean_img_min
        second_point = np.array(second_point) -  self.zone_size**2*mean_img_min
        third_point  =np.array(third_point) -  self.zone_size**2*mean_img_min
        print("The data is unpacked  --- %s seconds ---" % (t.time() - start_time))
        return first_point, second_point, third_point
        
    
            
    def max_research(self, point_data, interval):
        start = 0
        max_point_data = []
        point_data_len = len(point_data)
        while start + interval < point_data_len:
            informative_data = point_data[start:start + interval]
            max_point_data.append(np.max(informative_data))
            start += interval
        return np.array(max_point_data) 
          
        
        

    def path_res(self, path_to_result):
        if path_to_result is not None:
            if os.path.exists(path_to_result):
                print("This directory already exists. Do you want rewrite ? ")
                correct_answ = False
                while correct_answ is False:
                    user_answ = int(input("  YES - 1 / NO - 0"))
                    if user_answ != 0 and user_answ != 1:
                        print("Incorrect input")
                    else: 
                        user_answ = bool(user_answ)
                        correct_answ = True
                if user_answ:
                    self.del_rewrite(path_to_result)
                    return path_to_result
                else:
                    new_path = input("Write new path ")
                    self.create_folder(new_path)
                    return new_path
            else:
                self.create_folder(path_to_result)
                return path_to_result
        else:
            default_path  = os.getcwd() + "\\Mesurments"
            if os.path.exists(default_path): 
                print("This directory already exists. Do you want rewrite ? ")
                correct_answ = False
                while correct_answ is False:
                    user_answ = int(input("  YES - 1 / NO - 0 "))
                    if user_answ != 0 and user_answ != 1:
                        print("Incorrect input")
                    else: 
                        user_answ = bool(user_answ)
                        correct_answ = True
                if user_answ:
                    self.del_rewrite(default_path)
                    return default_path
                else:
                    i = 1
                    default_path = os.getcwd() + "\\Mesurments_" + str(i)
                    while os.path.exists(default_path):
                        c = os.getcwd() + "\\Mesurments_" + str(i)
                        i += 1
                    self.create_folder(default_path)
                    return default_path
            else:
                self.create_folder(default_path)
                return default_path
            
    def path_data(self, path_to_data):
        if path_to_data != None:
            return path_to_data 
        else:
            print("Incorrect path. Directory wasn't found ")
            exit()
                           
    def del_rewrite(self, path):
        try:
            shutil.rmtree(path)
        except OSError:
            print ("Failed to remove directory %s" % path)
            exit()
        else:
            print ("Directory %s was removed " % path)
            
        self.create_folder(path)
    

    def create_folder(self, path):
        try:
            os.mkdir(path)
        except OSError:
            print (" Failed to create directory %s " % path)
            exit()
        else:
            print ("Directory %s was created " % path)

IndentationError: expected an indented block (1547003256.py, line 220)

In [80]:
res = Measurment(
                path_to_data = "E:\\учёба\\7 семестр\\УИР и Практика 7 сем\\result_21_12_21_v1",
                zone_coordinates = ((12,251),(12,145),(12,40)), 
                zone_size = 20, 
                skiped_img = 650,
                
    )

This directory already exists. Do you want rewrite ? 
  YES - 1 / NO - 0 1
Directory E:\учёба\8 семестр\Science\Mesurments was removed 
Directory E:\учёба\8 семестр\Science\Mesurments was created 
The data is unpacked  --- 26.94771718978882 seconds ---


In [81]:
res.get_value(1500)

In [82]:
res.calculated_time_param

[[array([  104,  1646,  3162,  4645,  6145,  7645,  9145, 10645, 12145,
         13645, 15145, 16645, 18144, 19645, 21145, 22644, 24144, 25661,
         27178, 28661, 30177, 31661, 33161, 34677, 36160, 37677, 39160,
         40660, 42160, 43660, 45160, 46660, 48159, 49660]),
  array([  246,  1791,  3309,  4793,  6292,  7792,  9288, 10793, 12290,
         13792, 15290, 16789, 18289, 19792, 21291, 22789, 24290, 25808,
         27322, 28805, 30322, 31808, 33305, 34824, 36305, 37823, 39305,
         40807, 42307, 43806, 45307, 46804, 48304, 49804]),
  array([  628,  2127,  3644,  5128,  6627,  8127,  9627, 11127, 12627,
         14127, 15627, 17127, 18627, 20127, 21643, 23143, 24643, 26143,
         27643, 29159, 30643, 32159, 33642, 35159, 36642, 38159, 39659,
         41142, 42642, 44142, 45641, 47142, 48642, 50141]),
  array([  726,  2226,  3742,  5225,  6726,  8226,  9725, 11224, 12725,
         14225, 15724, 17225, 18724, 20224, 21741, 23240, 24741, 26241,
         27740, 29257, 30741